In [1]:
#Buscando Wiki data :)
#display(dbutils.fs.ls("dbfs:///databricks-datasets/wikipedia-datasets/data-001/en_wikipedia/perline-xml/"))

In [2]:
#para visualizar los datos, hay que crear un RDD a partir del XML
data = sc.textFile("dbfs:/databricks-datasets/wikipedia-datasets/data-001/en_wikipedia/perline-xml/enwiki-latest-pages-articles1.xml-p000000010p000010000.bz2-part-00000")
#data.take(2)

In [3]:
%sql
drop table wiki2

In [4]:
%sql
CREATE TABLE wiki2 (id string, text string)
USING com.databricks.spark.xml
OPTIONS (path "dbfs:/databricks-datasets/wikipedia-datasets/data-001/en_wikipedia/perline-xml/enwiki-latest-pages-articles1.xml-p000000010p000010000.bz2-part-00000", rowTag "revision")

In [5]:
wiki_df = spark.sql("select id, text from wiki2") #creación de dataFrame

In [6]:
#Creamos un DataFrame usando la tabla wiki2
wiki_df = spark.sql("select id, text from wiki2")
wiki_df.show()

+---------+--------------------+
 id| text|
+---------+--------------------+
631144794|#REDIRECT [[Compu...|
683845221|{{Redirect2|Anarc...|
 74466652|#REDIRECT [[Histo...|
407008307|#REDIRECT [[Geogr...|
616420354|#REDIRECT [[Demog...|
 74466499|#REDIRECT [[Commu...|
409266982|#REDIRECT [[Trans...|
558328133|#REDIRECT [[Afgha...|
 46448859|#REDIRECT [[Forei...|
 74466798|#REDIRECT [[Assis...|
627604809|#REDIRECT [[Amoeb...|
682116152|{{Hatnote|This ar...|
 74467016|#REDIRECT [[Histo...|
 74466817|#REDIRECT [[Demog...|
 74467061|#REDIRECT [[As_We...|
 74467128|#REDIRECT [[Polit...|
 74467158|#REDIRECT [[Econo...|
670440233|{{Other uses}} {{...|
308571542|#REDIRECT [[Afroa...|
626556798|#REDIRECT [[Const...|
+---------+--------------------+
only showing top 20 rows

In [7]:
#1. Convertimos todo el texto a minúscula y elimminamos caracteres especiales
from pyspark.sql.functions import lower, regexp_replace
wiki_lower = wiki_df.select('id', (lower(
  regexp_replace('text', "[^a-zA-Z\\s]", " ")).alias('text')))
wiki_lower.show()

+---------+--------------------+
 id| text|
+---------+--------------------+
631144794| redirect compu...|
683845221| redirect anarc...|
 74466652| redirect histo...|
407008307| redirect geogr...|
616420354| redirect demog...|
 74466499| redirect commu...|
409266982| redirect trans...|
558328133| redirect afgha...|
 46448859| redirect forei...|
 74466798| redirect assis...|
627604809| redirect amoeb...|
682116152| hatnote this ar...|
 74467016| redirect histo...|
 74466817| redirect demog...|
 74467061| redirect as we...|
 74467128| redirect polit...|
 74467158| redirect econo...|
670440233| other uses ...|
308571542| redirect afroa...|
626556798| redirect const...|
+---------+--------------------+
only showing top 20 rows

In [8]:
#2. Removemos espacios en blanco
import pyspark.sql.functions as F
from pyspark.sql.functions import col

def remove_all_whitespace(col):
    return F.regexp_replace(col, "\\s+", " ")
wiki_espacios = wiki_lower.withColumn("text_sin_esp",remove_all_whitespace(col("text")))
wiki_espacios.show()

+---------+--------------------+--------------------+
 id| text| text_sin_esp|
+---------+--------------------+--------------------+
631144794| redirect compu...| redirect compute...|
683845221| redirect anarc...| redirect anarchi...|
 74466652| redirect histo...| redirect history...|
407008307| redirect geogr...| redirect geograp...|
616420354| redirect demog...| redirect demogra...|
 74466499| redirect commu...| redirect communi...|
409266982| redirect trans...| redirect transpo...|
558328133| redirect afgha...| redirect afghan ...|
 46448859| redirect forei...| redirect foreign...|
 74466798| redirect assis...| redirect assisti...|
627604809| redirect amoeb...| redirect amoeba ...|
682116152| hatnote this ar...| hatnote this art...|
 74467016| redirect histo...| redirect history...|
 74466817| redirect demog...| redirect demogra...|
 74467061| redirect as we...| redirect as we m...|
 74467128| redirect polit...| redirect politic...|
 74467158| redirect econo...| redirect economy...|
670440233| other uses ...| other uses use d...|
308571542| redirect afroa...| redirect afroasi...|
626556798| redirect const...| redirect constru...|
+---------+--------------------+--------------------+
only showing top 20 rows

In [9]:
#3. Tokenizamos el texto
from pyspark.ml.feature import Tokenizer
tokenizer = Tokenizer(inputCol='text_sin_esp', outputCol='text_token')
wiki_token = tokenizer.transform(wiki_espacios).select('id', 'text_token')
wiki_token.show()

+---------+--------------------+
 id| text_token|
+---------+--------------------+
631144794|[, redirect, comp...|
683845221|[, redirect, anar...|
 74466652|[, redirect, hist...|
407008307|[, redirect, geog...|
616420354|[, redirect, demo...|
 74466499|[, redirect, comm...|
409266982|[, redirect, tran...|
558328133|[, redirect, afgh...|
 46448859|[, redirect, fore...|
 74466798|[, redirect, assi...|
627604809|[, redirect, amoe...|
682116152|[, hatnote, this,...|
 74467016|[, redirect, hist...|
 74466817|[, redirect, demo...|
 74467061|[, redirect, as, ...|
 74467128|[, redirect, poli...|
 74467158|[, redirect, econ...|
670440233|[, other, uses, u...|
308571542|[, redirect, afro...|
626556798|[, redirect, cons...|
+---------+--------------------+
only showing top 20 rows

In [10]:
from pyspark.ml.feature import StopWordsRemover
remover = StopWordsRemover()
stopwords = remover.getStopWords()
stopwords[:] #Revisamos la lista actual de Stopwords

Out[8]: ['i',
 'me',
 'my',
 'myself',
 'we',
 'our',
 'ours',
 'ourselves',
 'you',
 'your',
 'yours',
 'yourself',
 'yourselves',
 'he',
 'him',
 'his',
 'himself',
 'she',
 'her',
 'hers',
 'herself',
 'it',
 'its',
 'itself',
 'they',
 'them',
 'their',
 'theirs',
 'themselves',
 'what',
 'which',
 'who',
 'whom',
 'this',
 'that',
 'these',
 'those',
 'am',
 'is',
 'are',
 'was',
 'were',
 'be',
 'been',
 'being',
 'have',
 'has',
 'had',
 'having',
 'do',
 'does',
 'did',
 'doing',
 'a',
 'an',
 'the',
 'and',
 'but',
 'if',
 'or',
 'because',
 'as',
 'until',
 'while',
 'of',
 'at',
 'by',
 'for',
 'with',
 'about',
 'against',
 'between',
 'into',
 'through',
 'during',
 'before',
 'after',
 'above',
 'below',
 'to',
 'from',
 'up',
 'down',
 'in',
 'out',
 'on',
 'off',
 'over',
 'under',
 'again',
 'further',
 'then',
 'once',
 'here',
 'there',
 'when',
 'where',
 'why',
 'how',
 'all',
 'any',
 'both',
 'each',
 'few',
 'more',
 'most',
 'other',
 'some',
 'such',
 'no',
 'nor',
 'not',
 'only',
 'own',
 'same',
 'so',
 'than',
 'too',
 'very',
 's',
 't',
 'can',
 'will',
 'just',
 'don',
 'should',
 'now',
 "i'll",
 "you'll",
 "he'll",
 "she'll",
 "we'll",
 "they'll",
 "i'd",
 "you'd",
 "he'd",
 "she'd",
 "we'd",
 "they'd",
 "i'm",
 "you're",
 "he's",
 "she's",
 "it's",
 "we're",
 "they're",
 "i've",
 "we've",
 "you've",
 "they've",
 "isn't",
 "aren't",
 "wasn't",
 "weren't",
 "haven't",
 "hasn't",
 "hadn't",
 "don't",
 "doesn't",
 "didn't",
 "won't",
 "wouldn't",
 "shan't",
 "shouldn't",
 "mustn't",
 "can't",
 "couldn't",
 'cannot',
 'could',
 "here's",
 "how's",
 "let's",
 'ought',
 "that's",
 "there's",
 "what's",
 "when's",
 "where's",
 "who's",
 "why's",
 'would']

In [11]:
#4. Eliminamos stopwords
from pyspark.ml.feature import StopWordsRemover
stopwordList = ["redirect","uses", "http", "title", "www", "date", "web", "org","name","r","camelcase"] 
remover = StopWordsRemover(inputCol="text_token", outputCol="filtered", stopWords=stopwordList)
wiki_stop = remover.transform(wiki_token)
wiki_stop.show()

+---------+--------------------+--------------------+
 id| text_token| filtered|
+---------+--------------------+--------------------+
631144794|[, redirect, comp...|[, computer, acce...|
683845221|[, redirect, anar...|[, anarchist, ana...|
 74466652|[, redirect, hist...|[, history, of, a...|
407008307|[, redirect, geog...|[, geography, of,...|
616420354|[, redirect, demo...|[, demographics, ...|
 74466499|[, redirect, comm...|[, communications...|
409266982|[, redirect, tran...|[, transport, in,...|
558328133|[, redirect, afgh...|[, afghan, armed,...|
 46448859|[, redirect, fore...|[, foreign, relat...|
 74466798|[, redirect, assi...|[, assistive, tec...|
627604809|[, redirect, amoe...| [, amoeba, from]|
682116152|[, hatnote, this,...|[, hatnote, this,...|
 74467016|[, redirect, hist...|[, history, of, a...|
 74466817|[, redirect, demo...|[, demographics, ...|
 74467061|[, redirect, as, ...|[, as, we, may, t...|
 74467128|[, redirect, poli...|[, politics, of, ...|
 74467158|[, redirect, econ...|[, economy, of, a...|
670440233|[, other, uses, u...|[, other, use, dm...|
308571542|[, redirect, afro...|[, afroasiatic, l...|
626556798|[, redirect, cons...|[, constructed, l...|
+---------+--------------------+--------------------+
only showing top 20 rows

In [12]:
temp_table_name = "wiki_cleaned"
wiki_stop.createOrReplaceTempView(temp_table_name)

In [13]:
%sql
select count(*) from wiki_cleaned

count(1)
6269


In [14]:
%sql
select id,filtered from wiki_cleaned limit 1

id,filtered
631144794,"List(, computer, accessibility, redr, move, from, up)"


In [15]:
#Iniciamos con la función CountVectorizer
from pyspark.ml.feature import CountVectorizer
cv = CountVectorizer(inputCol="filtered", outputCol="rawFeatures", vocabSize = 10000)
model = cv.fit(wiki_stop)
result = model.transform(wiki_stop)
result.show()

+---------+--------------------+--------------------+--------------------+
 id| text_token| filtered| rawFeatures|
+---------+--------------------+--------------------+--------------------+
631144794|[, redirect, comp...|[, computer, acce...|(10000,[18,161,30...|
683845221|[, redirect, anar...|[, anarchist, ana...|(10000,[0,1,2,3,4...|
 74466652|[, redirect, hist...|[, history, of, a...|(10000,[1,18,82,3...|
407008307|[, redirect, geog...|[, geography, of,...|(10000,[1,18,376,...|
616420354|[, redirect, demo...|[, demographics, ...|(10000,[1,18,376,...|
 74466499|[, redirect, comm...|[, communications...|(10000,[3,18,376,...|
409266982|[, redirect, tran...|[, transport, in,...|(10000,[3,18,376,...|
558328133|[, redirect, afgh...|[, afghan, armed,...|(10000,[18,376,48...|
 46448859|[, redirect, fore...|[, foreign, relat...|(10000,[1,18,376,...|
 74466798|[, redirect, assi...|[, assistive, tec...|(10000,[18,376,53...|
627604809|[, redirect, amoe...| [, amoeba, from]|(10000,[18,376],[...|
682116152|[, hatnote, this,...|[, hatnote, this,...|(10000,[0,1,2,3,4...|
 74467016|[, redirect, hist...|[, history, of, a...|(10000,[1,18,82,3...|
 74466817|[, redirect, demo...|[, demographics, ...|(10000,[1,18,376,...|
 74467061|[, redirect, as, ...|[, as, we, may, t...|(10000,[9,18,59,3...|
 74467128|[, redirect, poli...|[, politics, of, ...|(10000,[1,18,376,...|
 74467158|[, redirect, econ...|[, economy, of, a...|(10000,[1,18,376,...|
670440233|[, other, uses, u...|[, other, use, dm...|(10000,[0,1,2,3,4...|
308571542|[, redirect, afro...|[, afroasiatic, l...|(10000,[18,360,37...|
626556798|[, redirect, cons...|[, constructed, l...|(10000,[7,18,35,3...|
+---------+--------------------+--------------------+--------------------+
only showing top 20 rows

In [16]:
from pyspark.ml.feature import HashingTF
ht = HashingTF(inputCol="filtered", outputCol="rawFeaturesTF")
result = ht.transform(wiki_stop)
result.show()

+---------+--------------------+--------------------+--------------------+
 id| text_token| filtered| rawFeaturesTF|
+---------+--------------------+--------------------+--------------------+
631144794|[, redirect, comp...|[, computer, acce...|(262144,[50920,10...|
683845221|[, redirect, anar...|[, anarchist, ana...|(262144,[14,15,58...|
 74466652|[, redirect, hist...|[, history, of, a...|(262144,[9639,934...|
407008307|[, redirect, geog...|[, geography, of,...|(262144,[9639,839...|
616420354|[, redirect, demo...|[, demographics, ...|(262144,[9639,117...|
 74466499|[, redirect, comm...|[, communications...|(262144,[93435,93...|
409266982|[, redirect, tran...|[, transport, in,...|(262144,[9724,934...|
558328133|[, redirect, afgh...|[, afghan, armed,...|(262144,[35122,10...|
 46448859|[, redirect, fore...|[, foreign, relat...|(262144,[9639,547...|
 74466798|[, redirect, assi...|[, assistive, tec...|(262144,[38068,10...|
627604809|[, redirect, amoe...| [, amoeba, from]|(262144,[101169,2...|
682116152|[, hatnote, this,...|[, hatnote, this,...|(262144,[29,170,5...|
 74467016|[, redirect, hist...|[, history, of, a...|(262144,[9639,133...|
 74466817|[, redirect, demo...|[, demographics, ...|(262144,[9639,117...|
 74467061|[, redirect, as, ...|[, as, we, may, t...|(262144,[50940,70...|
 74467128|[, redirect, poli...|[, politics, of, ...|(262144,[9639,133...|
 74467158|[, redirect, econ...|[, economy, of, a...|(262144,[9639,133...|
670440233|[, other, uses, u...|[, other, use, dm...|(262144,[150,170,...|
308571542|[, redirect, afro...|[, afroasiatic, l...|(262144,[101169,1...|
626556798|[, redirect, cons...|[, constructed, l...|(262144,[94533,10...|
+---------+--------------------+--------------------+--------------------+
only showing top 20 rows